# Thư viện

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Load data, Pre-processing

## Load data

In [2]:
#Loading a dataset
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to ~/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [12]:
#Check số lượng labels
num_classes = metadata.features['label'].num_classes
print(num_classes)

5


## Pre-processing

### Resize, rescale

In [3]:
IMG_SIZE = 180
#Định nghĩa hàm resizing và rescaling
resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE),
  tf.keras.layers.Rescaling(1./255)
])

### Augmentation


*   RandomFlip(): Lật hình. Mặc định lật lên-xuống và trái-phải.
*   RandomRotation(): Xoay hình. Tham số truyền vào là mức độ xoay hình. Tham số dương sẽ quay ngược chiều đồng hồ, số âm ngược. Cần truyền 2 giá trị trong một tuple, nếu chỉ truyền một số x, thì sẽ chuyển thành [-x, x]
*   RandomContrast(): Chỉnh độ tương phản. Truyền vào một số là mức độ chỉnh.
*   RandomCrop(): Cắt hình ở vị trí ngẫu nhiên. Truyền vào size của hình.
*   RandomHeight(), RandomWidth(): Chỉnh chiều cao, rộng. Truyền vào mức độ chỉnh.
*   RandomZoom(): Zoom hình. Height_factor là mức độ zoom theo chiều dọc. Width_factor là mức độ zoom chiều ngang, mặc định là None, khi đó chiều ngang sẽ zoom theo chiều dọc sao cho giữ được tỉ lệ ngang*dọc ban đầu. Số âm sẽ zoom in, số dương ngược lại.
*   RandomTranslation(): Dời vị trí hình. Truyền tham số là mức độ dời hình theo chiều dọc và ngang.





In [6]:
#Định nghĩa 2 augmentations là random flip và random random rotation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip(),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomContrast(0.3),
  tf.keras.layers.RandomCrop(70, 70),
  tf.keras.layers.RandomHeight(0.3),
  tf.keras.layers.RandomWidth(0.3),
  tf.keras.layers.RandomZoom(0.3),
  tf.keras.layers.RandomTranslation(0.3, 0.3)
])

In [7]:
batch_size = 128
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [8]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

# Model

## No Freeze (Trainable Params)

In [13]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

def create_model():

    pretrained_model = tf.keras.applications.VGG19(weights='imagenet', include_top=False)
    last_output = pretrained_model.output
    x = GlobalAveragePooling2D()(last_output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Dense(250, activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Dense(100, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=pretrained_model.input, outputs=outputs)
    
    return model

In [14]:
model = create_model()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

MC=tf.keras.callbacks.ModelCheckpoint(
    "best.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True, 
    verbose=1
)

In [15]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs, 
  callbacks=[MC]
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


23/23 [==============================] - ETA: 0s - loss: 1.9004 - accuracy: 0.2183
Epoch 1: val_accuracy improved from -inf to 0.24251, saving model to best.hdf5
23/23 [==============================] - 101s 4s/step - loss: 1.9004 - accuracy: 0.2183 - val_loss: 1.5933 - val_accuracy: 0.2425
Epoch 2/5
23/23 [==============================] - ETA: 0s - loss: 1.6084 - accuracy: 0.2408
Epoch 2: val_accuracy did not improve from 0.24251
23/23 [==============================] - 62s 3s/step - loss: 1.6084 - accuracy: 0.2408 - val_loss: 1.6031 - val_accuracy: 0.2425
Epoch 3/5
23/23 [==============================] - ETA: 0s - loss: 1.6019 - accuracy: 0.2483
Epoch 3: val_accuracy improved from 0.24251 to 0.35150, saving model to best.hdf5
23/23 [==============================] - 60s 3s/step - loss: 1.6019 - accuracy: 0.2483 - val_loss: 1.6029 - val_accuracy: 0.3515
Epoch 4/5
23/23 [==============================] - ETA: 0s - loss: 1.5948 - accuracy: 0.2742
Epoch 4: val_accuracy did not improve 

In [17]:
model.evaluate(test_ds)

print('Best model:')
model.load_weights("best.hdf5")
model.evaluate(test_ds)

3/3 [==============================] - 1s 418ms/step - loss: 1.6122 - accuracy: 0.1907
Best model:
3/3 [==============================] - 1s 420ms/step - loss: 1.6074 - accuracy: 0.2589


[1.6074011325836182, 0.25885558128356934]

## Freeze (Untrainable Params)

In [18]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

def create_model_freeze():

    pretrained_model = tf.keras.applications.VGG19(weights='imagenet', include_top=False)
    pretrained_model.trainable = False
    last_output = pretrained_model.output
    x = GlobalAveragePooling2D()(last_output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Dense(250, activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Dense(100, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=pretrained_model.input, outputs=outputs)
    
    return model

In [19]:
model_freeze = create_model_freeze()
model_freeze.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

MC_freeze=tf.keras.callbacks.ModelCheckpoint(
    "best_freeze.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True, 
    verbose=1
)

In [20]:
epochs=5
history_freeze = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs, 
  callbacks=[MC_freeze]
)

Epoch 1/5
23/23 [==============================] - ETA: 0s - loss: 1.5696 - accuracy: 0.2783
Epoch 1: val_accuracy improved from -inf to 0.36785, saving model to best_freeze.hdf5
23/23 [==============================] - 54s 2s/step - loss: 1.5696 - accuracy: 0.2783 - val_loss: 1.5541 - val_accuracy: 0.3678
Epoch 2/5
23/23 [==============================] - ETA: 0s - loss: 1.4890 - accuracy: 0.3423
Epoch 2: val_accuracy improved from 0.36785 to 0.43597, saving model to best_freeze.hdf5
23/23 [==============================] - 54s 2s/step - loss: 1.4890 - accuracy: 0.3423 - val_loss: 1.4362 - val_accuracy: 0.4360
Epoch 3/5
23/23 [==============================] - ETA: 0s - loss: 1.4354 - accuracy: 0.3488
Epoch 3: val_accuracy did not improve from 0.43597
23/23 [==============================] - 47s 2s/step - loss: 1.4354 - accuracy: 0.3488 - val_loss: 1.3958 - val_accuracy: 0.3515
Epoch 4/5
23/23 [==============================] - ETA: 0s - loss: 1.4159 - accuracy: 0.3529
Epoch 4: val_ac

In [21]:
model_freeze.evaluate(test_ds)

print('Best model:')
model_freeze.load_weights("best_freeze.hdf5")
model_freeze.evaluate(test_ds)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


3/3 [==============================] - 2s 455ms/step - loss: 1.6426 - accuracy: 0.1880
Best model:
3/3 [==============================] - 1s 436ms/step - loss: 1.4390 - accuracy: 0.3542


[1.4389917850494385, 0.35422343015670776]

So sánh giữa việc freeze và unfreeze pre_trained model, với kết quả có được phía trên. Có thể thấy, với số epoch được chọn:

*   Khi train: Freeze cho kết quả tốt hơn unfreeze.
*   Khi test: Freeze cho kết quả khá tương đơn unfreeze.

